# [1978. Employees Whose Manager Left the Company](https://leetcode.com/problems/employees-whose-manager-left-the-company/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Employees

<pre>+-------------+----------+
| Column Name | Type     |
+-------------+----------+
| employee_id | int      |
| name        | varchar  |
| manager_id  | int      |
| salary      | int      |
+-------------+----------+</pre>
In SQL, employee_id is the primary key for this table.
This table contains information about the employees, their salary, and the ID of their manager. Some employees do not have a manager (manager_id is null). 
 

Find the IDs of the employees whose salary is strictly less than dollar 30000 and whose manager left the company. When a manager leaves the company, their information is deleted from the Employees table, but the reports still have their manager_id set to the manager that left.

Return the result table ordered by employee_id.

The result format is in the following example.

 

Example 1:

Input:  
Employees table:
<pre>+-------------+-----------+------------+--------+
| employee_id | name      | manager_id | salary |
+-------------+-----------+------------+--------+
| 3           | Mila      | 9          | 60301  |
| 12          | Antonella | null       | 31000  |
| 13          | Emery     | null       | 67084  |
| 1           | Kalel     | 11         | 21241  |
| 9           | Mikaela   | null       | 50937  |
| 11          | Joziah    | 6          | 28485  |
+-------------+-----------+------------+--------+</pre>
Output: 
<pre>+-------------+
| employee_id |
+-------------+
| 11          |
+-------------+</pre>

Explanation: 
The employees with a salary less than $30000 are 1 (Kalel) and 11 (Joziah).
Kalel's manager is employee 11, who is still in the company (Joziah).
Joziah's manager is employee 6, who left the company because there is no row for employee 6 as it was deleted.

In [1]:
#pandas schema
import pandas as pd

data = [[3, 'Mila', 9, 60301], [12, 'Antonella', None, 31000], [13, 'Emery', None, 67084], [1, 'Kalel', 11, 21241],
        [9, 'Mikaela', None, 50937], [11, 'Joziah', 6, 28485]]
employees = pd.DataFrame(data, columns=['employee_id', 'name', 'manager_id', 'salary']).astype(
    {'employee_id': 'Int64', 'name': 'object', 'manager_id': 'Int64', 'salary': 'Int64'})

# spark has issues with null in Int64 of pandas, hence converting to str for now. Will convert to int after loading in spark.
employees['manager_id'] = employees['manager_id'].astype('str')

In [2]:
employees

,employee_id,name,manager_id,salary
0,3,Mila,9,60301
1,12,Antonella,<NA>,31000
2,13,Emery,<NA>,67084
3,1,Kalel,11,21241
4,9,Mikaela,<NA>,50937
5,11,Joziah,6,28485


In [3]:
#pyspark schema

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

employees_df = spark.createDataFrame(employees)
employees_df.withColumn('manager_id', F.col('manager_id').cast('int')).show()

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/05 14:49:36 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/05 14:49:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 14:49:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----------+---------+----------+------+
|employee_id|     name|manager_id|salary|
+-----------+---------+----------+------+
|          3|     Mila|         9| 60301|
|         12|Antonella|      null| 31000|
|         13|    Emery|      null| 67084|
|          1|    Kalel|        11| 21241|
|          9|  Mikaela|      null| 50937|
|         11|   Joziah|         6| 28485|
+-----------+---------+----------+------+


In [4]:
employees_df.printSchema()

root
 |-- employee_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- manager_id: string (nullable = true)
 |-- salary: long (nullable = true)


In [5]:
# Solving in pyspark dataframe

employees_df \
    .filter(F.col('salary') < 30000) \
    .join(employees_df.alias('b').select('employee_id'), on=F.col('b.employee_id') == F.col('manager_id'), how='anti') \
    .select('employee_id') \
    .orderBy('employee_id') \
    .show()

+-----------+
|employee_id|
+-----------+
|         11|
+-----------+


In [6]:
# another way

employee_list = employees_df.select('employee_id').rdd.map(lambda x: x[0]).collect()

employees_df \
    .filter((F.col('salary') < 30000) & (~F.col('manager_id').isin(employee_list))) \
    .select('employee_id') \
    .orderBy('employee_id') \
    .show()

+-----------+
|employee_id|
+-----------+
|         11|
+-----------+


In [7]:
# In Spark SQL

employees_df.createOrReplaceTempView('employees')

spark.sql('''
select employee_id
from employees
where salary<30000 and not manager_id in (select distinct employee_id from employees)
order by employee_id;
''').show()

+-----------+
|employee_id|
+-----------+
|         11|
+-----------+


In [8]:
spark.stop()